In [1]:
import pandas as pd
from apyori import apriori
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder
df = pd.read_csv('compras.csv', header=None)
df.head()

C:\Users\1-22-10504\AppData\Local\Temp\ipykernel_13948\1136154338.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,cliente,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
1,JÚLIO DEMARQUE,limão,peito de peru,manga,cerveja,picanha,carvão,sal,linguiça,mel,geleia,cereal,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,MAYCON SOARES,salsicha,cerveja,picanha,catchup,geleia,cebola,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,WEMERSON NETO,queijo,frutas,chá,alface,granola,feijão,farinha de trigo,suco de laranja,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,JOÃO DEMARQUE,pimenta,sardinha enlatada,barra de cereal,molho de tomate,gelatina,vinho branco,pão,tempero pronto,vinho tinto,presunto,arroz,feijão,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
# pegando uniques
uniques = df[0].unique()
print(uniques)

['cliente' 'JÚLIO DEMARQUE' 'MAYCON SOARES' 'WEMERSON NETO'
 'JOÃO DEMARQUE' 'MAYCON SANTOS' 'PEDRO SOARES' 'OTÁVIO NETO'
 'VICTOR NETO' 'GABRIEL NETO' 'MAYCON OLIVEIRA' 'MAYCON SILVA'
 'MAYCON RODRIGUES' 'GABRIEL SANTOS' 'LUIZ OLIVEIRA' 'WEMERSON SANTOS'
 'PEDRO CARDOSO' 'JÚLIO RODRIGUES' 'SÁVIO SOARES' 'WEMERSON SOARES'
 'VICTOR RODRIGUES' 'FILIPE LIMA' 'MAYCON DEMARQUE' 'ALYSON SILVA'
 'OTÁVIO CARDOSO' 'JOÃO BANDEIRA' 'ALYSON SOARES' 'GABRIEL CARDOSO'
 'SERGIO CARDOSO' 'SÁVIO CARDOSO' 'JÚLIO SILVA' 'ALYSON LIMA'
 'GABRIEL NÁPOLES' 'SERGIO SILVA' 'JOÃO OLIVEIRA' 'GABRIEL BANDEIRA'
 'VICTOR COSTA' 'SÁVIO NETO' 'LUIZ SILVA' 'JÚLIO BANDEIRA' 'GUSTAVO CRUZ'
 'OTÁVIO MASSI' 'DANIEL CRUZ' 'LUIZ SOARES' 'DANIEL SILVA' 'DAVI SILVA'
 'SÁVIO CRUZ' 'WEMERSON RODRIGUES' 'SERGIO BANDEIRA' 'GABRIEL SOARES'
 'FILIPE CARDOSO' 'GABRIEL CRUZ' 'LUIZ CARDOSO' 'VANESSA SANTOS'
 'PEDRO COSTA' 'JÚLIO MASSI' 'GABRIEL SILVA' 'DANIEL OLIVEIRA'
 'DAVI NÁPOLES' 'WEMERSON CRUZ' 'GABRIEL RODRIGUES' 'VANESSA MASSI

In [3]:
# itens com mais compras
itens_mais_comprados = df.stack().value_counts()
# 20 itens mais comprados
itens_mais_comprados.head(20)

queijo               1241
presunto             1129
pão                  1056
feijão               1040
picanha               869
arroz                 856
salada                837
manteiga              803
cerveja               759
leite                 722
bolo                  716
salsicha              682
açúcar                631
sorvete               621
alface                618
carvão                612
batata frita          607
cerveja artesanal     581
tomate                577
refrigerante          573
Name: count, dtype: int64

In [7]:
transactions = []
for i in range(0,df.shape[0]):
    transactions.append([str(df.values[i,j])for j in range(0,df.shape[1])])
    regras = apriori(transactions=transactions, min_support=0.003, min_confidence=0.2, min_lift=3, min_length=2, max_length=2)
resultados = list(regras)
resultados

resultados_organizados = sorted(resultados, key=lambda x: x.ordered_statistics[0].lift, reverse=True)

for i, resultado in enumerate(resultados_organizados[:10], start=1):
    print(f"Regra {i}: {resultado}")

Regra 1: RelationRecord(items=frozenset({'refrigerante', 'pizza'}), support=0.024297570242975703, ordered_statistics=[OrderedStatistic(items_base=frozenset({'pizza'}), items_add=frozenset({'refrigerante'}), confidence=0.6214833759590793, lift=11.059528902076071), OrderedStatistic(items_base=frozenset({'refrigerante'}), items_add=frozenset({'pizza'}), confidence=0.43238434163701067, lift=11.059528902076071)])
Regra 2: RelationRecord(items=frozenset({'batata frita', 'hambúrguer'}), support=0.022897710228977103, ordered_statistics=[OrderedStatistic(items_base=frozenset({'batata frita'}), items_add=frozenset({'hambúrguer'}), confidence=0.38166666666666665, lift=10.96852969348659), OrderedStatistic(items_base=frozenset({'hambúrguer'}), items_add=frozenset({'batata frita'}), confidence=0.6580459770114943, lift=10.96852969348659)])
Regra 3: RelationRecord(items=frozenset({'açúcar', 'café'}), support=0.023797620237976203, ordered_statistics=[OrderedStatistic(items_base=frozenset({'açúcar'}), i

In [5]:
df_compras_lista = df.apply(lambda x: x.dropna().tolist(), axis=1).tolist()

te = TransactionEncoder()
te_ary = te.fit(df_compras_lista).transform(df_compras_lista)

# Converte o array resultante em um DataFrame pandas
df_onehot = pd.DataFrame(te_ary, columns=te.columns_)

# Agora você pode aplicar o fpgrowth no DataFrame one-hot
frequent_itemsets = fpgrowth(df_onehot, min_support=0.003,use_colnames=True)
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=3)
rules.size

14540

In [8]:
# imprimindo as regras mais importantes ordenadas por lift
rules = rules.sort_values(by='lift', ascending=False)
rules.head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
687,"(catchup, hambúrguer)","(cerveja, mostarda, batata frita)",0.014499,0.005599,0.0041,0.282759,50.497660,0.004018,1.386424,0.994618
674,"(cerveja, mostarda, batata frita)","(catchup, hambúrguer)",0.005599,0.014499,0.0041,0.732143,50.497660,0.004018,3.679205,0.985717
685,"(hambúrguer, mostarda)","(cerveja, catchup, batata frita)",0.010999,0.007399,0.0041,0.372727,50.373587,0.004018,1.582407,0.991049
676,"(cerveja, catchup, batata frita)","(hambúrguer, mostarda)",0.007399,0.010999,0.0041,0.554054,50.373587,0.004018,2.217760,0.987455
1031,"(salada, azeite)","(tomate, cenoura, vinagre)",0.015098,0.005399,0.0040,0.264901,49.060584,0.003918,1.353015,0.994635
1030,"(tomate, cenoura, vinagre)","(salada, azeite)",0.005399,0.015098,0.0040,0.740741,49.060584,0.003918,3.798906,0.984935
1247,"(tomate, azeite)","(salada, alface, vinagre)",0.011099,0.007799,0.0042,0.378378,48.514900,0.004113,1.596149,0.990380
1238,"(salada, alface, vinagre)","(tomate, azeite)",0.007799,0.011099,0.0042,0.538462,48.514900,0.004113,2.142619,0.987086
1026,"(salada, cenoura, vinagre)","(tomate, azeite)",0.007599,0.011099,0.0040,0.526316,47.420578,0.003915,2.087680,0.986408
1035,"(tomate, azeite)","(salada, cenoura, vinagre)",0.011099,0.007599,0.0040,0.360360,47.420578,0.003915,1.551500,0.989899
